# Feature Engineering
# Klassifikation
# Regression
# Validierung und mehr
# Deployment und Abschluss

In [2]:
## prepare environment
import pandas as pd
import numpy as np
datapath = '../3_data'
from os import chdir; chdir(datapath)

## Das finale Modell
## Feature Engineering in der Produktion
### Missing Values
### Neue Kategorien
### Protokollieren

In [4]:
data = pd.read_csv('bank_data.csv', sep=';')

import datetime

f = open('fe_prod_log.log','w')
f.write(datetime.datetime.now().strftime("[%Y-%m-%d %H:%M:%S] (timestamp)"))

f.write('\n\n')

s = data.isna().sum()
f.write('features with NA\'s\n')
f.write('=======================')

f.write('\n')
f.write(s[s > 0].to_string())
f.write('\n\n')

## value counts of not numeric features
f.write('categorical cols levels\n')
f.write('=======================')
f.write('\n')
catcolnames = data.select_dtypes(include='object').columns
for ccn in catcolnames:
    f.write(ccn)
    f.write('\n')
    f.write(data[ccn].value_counts().sort_index().to_string())
    f.write('\n\n')
f.close() 

## Modellübergabe in die Prodkution

### Modelle speichern scikit-learn intern

In [7]:
## load data
from bfh_cas_pml import prep_data
X_train, X_test, y_train, y_test = prep_data('melb_data_prep.csv', 'Price')

In [8]:
## three models:

## StandardScaler
from sklearn.preprocessing import StandardScaler
model_sc = StandardScaler().fit(X_train)

## LinearRegression
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression().fit(X_train, y_train)

## DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
model_dt = DecisionTreeRegressor(max_depth= 2, random_state=1234).fit(X_train, y_train)

In [9]:
## save models
import pickle 
with open('model_sc.pkl', 'wb') as pickle_file:
    pickle.dump(model_sc, pickle_file)
with open('model_lr.pkl', 'wb') as pickle_file:
    pickle.dump(model_lr, pickle_file)
with open('model_dt.pkl', 'wb') as pickle_file:
    pickle.dump(model_dt, pickle_file)

In [10]:
## reload models
with open('model_sc.pkl', 'rb') as pickle_file:
    model_sc_2 = pickle.load(pickle_file)
with open('model_lr.pkl', 'rb') as pickle_file:
    model_lr_2 = pickle.load(pickle_file)
with open('model_dt.pkl', 'rb') as pickle_file:
    model_dt_2 = pickle.load(pickle_file)  

In [11]:
## compare model_sc
print(model_sc.mean_, '\n')
print(model_sc.var_, '\n')
print(model_sc_2.mean_, '\n')
print(model_sc_2.var_, '\n')

[ 2.93777524e+00  1.45457511e+00  1.04163676e+01  1.43785679e+00
  1.69132279e+00  2.36783112e+00  2.09905428e+00  1.96821913e+03
  5.74995922e+00 -3.78088068e+01  1.44997459e+02  7.51771163e+03
  6.57233730e-01  1.29750449e-01  9.23177296e-02  2.90817159e-01
  3.75958245e-02  3.40890556e-01  1.14173870e-02  2.10569238e-01
  7.16408416e+00  2.01656288e+03  4.83420323e+00] 

[9.20202455e-01 4.69760099e-01 3.70020409e+01 4.29958153e-01
 7.49360361e-01 7.18266598e-01 3.44600718e-02 6.37436323e+02
 3.80159334e+01 5.49566645e-03 9.22692994e-03 2.02826662e+07
 2.25277554e-01 1.12915270e-01 8.37951664e-02 2.06242539e-01
 3.61823785e-02 2.24684185e-01 1.12870303e-02 1.66229834e-01
 6.22686859e+00 2.46046460e-01 1.25085102e+00] 

[ 2.93777524e+00  1.45457511e+00  1.04163676e+01  1.43785679e+00
  1.69132279e+00  2.36783112e+00  2.09905428e+00  1.96821913e+03
  5.74995922e+00 -3.78088068e+01  1.44997459e+02  7.51771163e+03
  6.57233730e-01  1.29750449e-01  9.23177296e-02  2.90817159e-01
  3.75958

In [12]:
## compare model_lr
print(model_lr.intercept_)
print(model_lr.coef_, '\n')
print(model_lr_2.intercept_)
print(model_lr_2.coef_)

-118270989.87425248
[ 2.50834327e+05 -1.45871504e+05 -4.13257480e+04  1.54116175e+05
  3.88792264e+04  8.20265759e+04  2.75423073e+05 -2.32056094e+03
 -5.38944769e+03 -5.85782602e+05  1.77374502e+05 -1.44436659e+00
  7.90575463e+04  2.88647655e+04  3.61587994e+04 -1.90495866e+05
  7.08524933e+04  2.24267526e+05  2.21844422e+05 -2.42468838e+05
  2.75413428e+03  3.71147038e+04  2.86865125e+03] 

-118270989.87425248
[ 2.50834327e+05 -1.45871504e+05 -4.13257480e+04  1.54116175e+05
  3.88792264e+04  8.20265759e+04  2.75423073e+05 -2.32056094e+03
 -5.38944769e+03 -5.85782602e+05  1.77374502e+05 -1.44436659e+00
  7.90575463e+04  2.88647655e+04  3.61587994e+04 -1.90495866e+05
  7.08524933e+04  2.24267526e+05  2.21844422e+05 -2.42468838e+05
  2.75413428e+03  3.71147038e+04  2.86865125e+03]


In [13]:
## compare model_dt
from sklearn.tree import export_text
print(export_text(
    model_dt, feature_names=list(X_train.columns)))
print(export_text(
    model_dt_2, feature_names=list(X_train.columns)))

|--- Rooms <= 3.50
|   |--- Type <= 1.50
|   |   |--- value: [1063330.72]
|   |--- Type >  1.50
|   |   |--- value: [683338.07]
|--- Rooms >  3.50
|   |--- Regionname_Southern_Metropolitan <= 0.50
|   |   |--- value: [1169298.15]
|   |--- Regionname_Southern_Metropolitan >  0.50
|   |   |--- value: [2133427.66]

|--- Rooms <= 3.50
|   |--- Type <= 1.50
|   |   |--- value: [1063330.72]
|   |--- Type >  1.50
|   |   |--- value: [683338.07]
|--- Rooms >  3.50
|   |--- Regionname_Southern_Metropolitan <= 0.50
|   |   |--- value: [1169298.15]
|   |--- Regionname_Southern_Metropolitan >  0.50
|   |   |--- value: [2133427.66]



### Modelle speichern extern mit PMML

**Vorbereitungen:**  
* Java Runtime muss vorhanden sein
* Library installieren direkt in Notebook:  
`!pip install sklearn2pmml`

In [16]:
## import libraries
from sklearn.pipeline import Pipeline
from sklearn2pmml import PMMLPipeline, sklearn2pmml

In [17]:
## StandardScaler
from sklearn.preprocessing import StandardScaler
pipeline = PMMLPipeline([("scaler", StandardScaler())]).fit(X_train)
sklearn2pmml(pipeline, "StandardScaler_melb.pmml", with_repr = True)

In [18]:
## LinearRegression
pipeline = PMMLPipeline([('regressor', LinearRegression())]).fit(X_train, y_train)
sklearn2pmml(pipeline, "LinearRegression_melb.pmml", with_repr = True)

In [19]:
## DecisionTreeClassifier
pipeline = PMMLPipeline([('regressor', DecisionTreeRegressor(max_depth= 2, random_state=1234))]).fit(X_train, y_train)
sklearn2pmml(pipeline, "DecisionTreeRegressor_melb.pmml", with_repr = True)

* re-import a pmml model
  * see https://stackoverflow.com/questions/52393301/use-pmml-models-in-python

## Der Modellierungsprozess

(kein Code unter diesem Titel)

## pyCaret

vgl. `5.5 Deployment und Abschluss - pycaret.ipynb`